In [17]:
import numpy as np
import healpy as hp
import pyccl
from astropy.io import fits
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.special import eval_legendre
from joblib import Parallel, delayed
import pickle

In [18]:
nside = 2048
lmax = 2 * nside

In [19]:
mock_base_dir = '/spiff/pierfied/Simulations/des_y3_mocks/'

hdu = fits.open(mock_base_dir + '2pt_NG_final_2ptunblind_02_26_21_wnz_maglim_covupdate.fits')
data = hdu['nz_source'].data
y3_z_low = data['Z_LOW']
y3_z_mid = data['Z_MID']
y3_z_high = data['Z_HIGH']
num_bins = 4
y3_nz = np.zeros([num_bins, len(y3_z_mid)])
for i in range(num_bins):
    y3_nz[i] = data[f'BIN{i+1}']
y3_nz[:,-1] = 0

dz = y3_z_high - y3_z_low
y3_nz /= np.sum(y3_nz * dz[None,:], axis=-1)[:,None]

In [20]:
cosmo = pyccl.Cosmology(
    Omega_c=0.233,
    Omega_b=0.046,
    h=0.7,
    sigma8=0.82,
    n_s=0.97
)

tracer = [pyccl.tracers.WeakLensingTracer(cosmo, (y3_z_mid, y3_nz[i])) for i in range(num_bins)]

ell = np.arange(lmax+1)
cl = np.array([pyccl.cls.angular_cl(cosmo, tracer[i], tracer[j], ell) for i in range(num_bins) for j in range(i, num_bins)])
pw = hp.pixwin(nside, lmax=lmax)
cl *= pw[None,:]**2

In [21]:
nmocks = 10
sim_cl = []
k_samps = []
for i in trange(nmocks):
    mock = np.load(mock_base_dir + f'mocks/mock_{i}.npy')[:,0,:]
    alm = hp.map2alm(mock, lmax=lmax, pol=False)
    sim_cl.append([hp.alm2cl(alm[j], alm[k]) for j in range(num_bins) for k in range(j, num_bins)])
    samp_inds = np.random.choice(mock.shape[-1], 10000, replace=False)
    k_samps.append(mock[:,samp_inds])
sim_cl = np.array(sim_cl)
k_samps = np.array(k_samps)

  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
fit_cl = np.copy(cl)
for i in range(len(cl)):
    p = np.polyfit(np.arange(lmax+1), (cl / sim_cl.mean(axis=0))[i,:], 3)
    fit_cl[i] /= np.polyval(p, np.arange(lmax+1))

In [23]:
shift = np.zeros(num_bins)
for i in range(num_bins):
    shift[i] = -lognorm.fit(k_samps[:,i,:].ravel())[1]

In [24]:
def compute_lognorm_cl_at_ell(mu, w, integrand, ell):
    integrand = np.log(np.polynomial.legendre.legval(mu, integrand) + 1)
    return 2 * np.pi * np.sum(w * integrand * eval_legendre(ell, mu))
    
def compute_lognorm_cl(cl, shift1, shift2, mu=None, w=None, order=2):
    lmax = len(cl) - 1
    lognorm_cl = np.zeros_like(cl)
    
    if mu is None or w is None:
        mu, w = np.polynomial.legendre.leggauss(order * lmax)
    
    integrand = ((2 * np.arange(lmax + 1) + 1) * cl / (4 * np.pi * shift1 * shift2))
    
    lognorm_cl = np.array(Parallel(n_jobs=-1)(delayed(compute_lognorm_cl_at_ell)(mu, w, integrand, ell) for ell in range(lmax+1)))
        
    return lognorm_cl

In [25]:
y_cl = []
y_mu = np.zeros(num_bins)

ind = 0
order = 2
mu, w = np.polynomial.legendre.leggauss(order * lmax)
with tqdm(total=cl.shape[0]) as pbar:
    for i in range(num_bins):
        xi0 = np.polynomial.legendre.legval(1, (2 * np.arange(lmax+1) + 1) * fit_cl[ind]) / (4 * np.pi)
        sigma2 = np.log(xi0 / (shift[i]**2) + 1)
        y_mu[i] = np.log(shift[i]) - 0.5 * sigma2

        for j in range(i, num_bins):
            y_cl.append(compute_lognorm_cl(fit_cl[ind], shift[i], shift[j], mu, w))
            ind += 1
            pbar.update()
y_cl = np.array(y_cl)

  0%|          | 0/10 [00:00<?, ?it/s]

In [26]:
lognorm_params = {'cl': fit_cl, 'shift': shift, 'y_cl': y_cl, 'mu': y_mu}
pickle.dump(lognorm_params, open('lognorm_params.pkl', 'wb'))